In [1]:
import os
import numpy as np 
import pandas as pd 

import tweepy
from konlpy.tag import Okt, Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
from hanspell import spell_checker

from pprint import pprint
import re
from math import log # IDF 계산을 위해

import warnings
warnings.filterwarnings(action='ignore')

## movie_summary

In [2]:
current_dir = os.getcwd()
movies = pd.read_csv(current_dir + '/data/movies_data.csv')
movie_genre = pd.read_csv(current_dir + '/data/movie_genre.csv')

,genre,script
0,액션,"액션 영화(action film)는 영화의 한 갈래로, 등장인물의 육체적인 움직임을..."
1,코미디,"코미디 영화(comedy film)는 유머에 중점을 둔 영화 장르로, 관객들로부터 ..."
2,다큐멘터리,"다큐멘터리 영화(영어: Documentary Film), 또는 기록 영화(記錄映畵,..."
3,판타지,판타지 영화란 판타지적인 내용을 영화로 만든 것이다.
4,공포,"공포 영화(恐怖映畫, 영어: horror film)란 귀신이나 유령 등 무서운 것과..."


In [ ]:
movies.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)
movies_summary = movies[['movie_id', 'summary']]

In [ ]:
join_table = pd.merge(movie_genre, movies_summary, on='movie_id', how='left')

In [ ]:
genre_summary = pd.DataFrame(join_table.groupby(['genre_id'])['summary'].sum().reset_index())

In [ ]:
drop_idx = join_table[join_table['summary'].str.contains(r'e')].index
join_table.drop(drop_idx, inplace=True)

In [ ]:
get_30 = lambda x: x[:30]

In [ ]:
grouped = join_table.groupby(['genre_id'])
grouped.size()

genre_id
0     412
1     484
2      57
3     223
4     161
5      53
6     298
7      32
8      18
10    108
11    247
12    849
13     79
14    107
15    518
16    174
17     46
dtype: int64

> genre_id 9(Made in Europe), 18(Reality TV) 없음

In [ ]:
selected_summary = grouped.apply(get_30).reset_index(drop=True)

In [ ]:
genre_summary = selected_summary[['genre_id', 'summary']]
genre_summary.head()

,genre_id,summary
0,0,유엔 가입을 위해 다수의 투표권을 지닌 아프리카 대륙에서 열심히 외교 활동을 벌이던...
1,0,진실을 위해서라면 몸을 사리지 않고 사회의 부조리를 취재하는 정의로운 열혈 기자 에...
2,0,평범하고 내성적인 고등학생 피터 파커는 우연히 방사능에 감염된 거미에 물린다.그 후...
3,0,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...
4,0,토니 스타크의 제안을 받고 시빌 워에 참여한 피터 파커는 다음 임무가 언제일지 설렘...


In [ ]:
genre_summary.shape

(498, 2)

In [ ]:
genre_summary = pd.DataFrame(genre_summary.groupby(['genre_id'])['summary'].sum().reset_index(drop=True))

In [ ]:
genre_summary.head()

,summary
0,유엔 가입을 위해 다수의 투표권을 지닌 아프리카 대륙에서 열심히 외교 활동을 벌이던...
1,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...
2,"권력의 정점에 서고픈 검사, 무소불위 권력을 쥐고 폼 나게 살고 싶었던 태수는 우여..."
3,평범하고 내성적인 고등학생 피터 파커는 우연히 방사능에 감염된 거미에 물린다.그 후...
4,30일마다 랜덤으로 레벨이 바뀌는 극한 생존의 수직 감옥 플랫폼. 최상위 레벨 0에...


## 키워드 추출

In [ ]:

FEATURE_POS = ['VA+ETM']# ["Noun", "Verb", "Adverb", "Adjective"]

def text_tokenizing(doc):
    documents = []
    tokenizer = Mecab()
    text_pos = [pair for pair in tokenizer.pos(doc) if pair[0] not in stop_words and len(pair[0]) > 1]
    words = []
    # return pprint(text_pos)
    
    for word, pos in text_pos:
        if pos in FEATURE_POS:
            words.append(word)

    total_words = ' '.join(words)
    return total_words

In [ ]:
    
def va_preprocessing(corpus):
    kor_corpus = [extract_kor(doc) for doc in corpus]
    # print(kor_corpus)
    ko_tokens = [text_tokenizing(doc) for doc in contents]
    ko_tokens = ''.join(ko_tokens)
    return kor_tokens

In [ ]:
ko_tokens = [''.join(ko_tokens) for token in ko_tokens]

In [ ]:
for L in kor_tokens:
    kor_total_tokens += L

In [ ]:
vocab = list(set(kor_total_tokens))
vocab.sort()

In [ ]:
result = []
N = len(kor_tokens) 

tf_ = pd.DataFrame(result, columns = vocab)
tf_.head()
# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([])
  d = kor_tokens[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_.head()

,가쁜,감미로운,강한,거센,거친,귀한,급한,기나긴,껄끄러운,끈질긴,...,즐거운,차가운,차디찬,착한,촌스러운,친한,커다란,파란,푸른,힘든
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_.head()

,IDF
가쁜,2.140066
감미로운,1.446919
강한,0.435318
거센,0.887303
거친,1.446919


In [ ]:
result = []
for i in range(N):
  result.append([])
  d = kor_tokens[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)

In [ ]:
df = tfidf_.transpose()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
가쁜,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.140066,0.000000
감미로운,0.0,1.446919,0.0,0.000000,0.0,1.446919,0.000000,0.000000,0.000000,0.000000,0.000000,1.446919,0.0,0.000000,0.000000,0.000000,0.000000
강한,0.0,0.435318,0.0,0.435318,0.0,0.435318,0.435318,0.435318,0.000000,0.435318,0.435318,0.435318,0.0,0.000000,0.000000,0.435318,0.435318
거센,0.0,0.000000,0.0,0.887303,0.0,0.000000,0.000000,0.000000,0.887303,1.774606,0.887303,0.000000,0.0,0.000000,0.887303,0.887303,0.000000
거친,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.446919,2.893838,0.000000,0.000000,0.0,1.446919,0.000000,0.000000,0.000000


In [ ]:
df.columns = ['액션', '코미디', '다큐멘터리', '판타지', '공포', '음악', '로맨스', '스포츠', '서부','애니메이션', '범죄', '드라마', '역사', '가족', '스릴러', 'SF', '전쟁']

In [ ]:
for genre in df.columns:
    print('========= %s =========' % genre)
    print(df[genre].sort_values(ascending=False).head(15))
    print(df[df[genre]>0][genre].sort_values(ascending=False))

========= 액션 =========
안타까운    1.446919
무서운     1.223775
엄청난     1.044920
멋진      0.753772
어릴      0.753772
어린      0.696613
낯선      0.194156
어설픈     0.000000
새로운     0.000000
색다른     0.000000
숱한      0.000000
아름다운    0.000000
아쉬운     0.000000
아픈      0.000000
어두운     0.000000
Name: 액션, dtype: float64
안타까운    1.446919
무서운     1.223775
엄청난     1.044920
멋진      0.753772
어릴      0.753772
어린      0.696613
낯선      0.194156
Name: 액션, dtype: float64
========= 코미디 =========
예쁜      2.447551
나쁜      2.140066
빛바랜     1.734601
아름다운    1.507544
감미로운    1.446919
뜨거운     1.223775
힘든      1.041454
어릴      0.753772
멋진      0.753772
뛰어난     0.753772
엄청난     0.696613
강한      0.435318
낯선      0.388312
어린      0.348307
아쉬운     0.000000
Name: 코미디, dtype: float64
예쁜      2.447551
나쁜      2.140066
빛바랜     1.734601
아름다운    1.507544
감미로운    1.446919
뜨거운     1.223775
힘든      1.041454
뛰어난     0.753772
멋진      0.753772
어릴      0.753772
엄청난     0.696613
강한      0.435318
낯선      0.388312
어린      0.348307
Name: 코미디,